In [ ]:
#TODO
#1. Add the 64*64 outputs to one image.
#2. For every pixel, convert it into the most probable one.Maybe depend on neighbours??
# Could be multiple overlaps, find a way to handle that.
#3. Convert every pixel to correct polygon json format

In [ ]:
# Every 64*64 is an probility distribution list/tensor of len 5 as well as top left corner of the image.
#input is atleast 256 images
def post_process(outputs,gamma=0.5):
    #outputs is a list of (64*64) tensors with 5 probabilities and a top left corner.
    #outputs = [(x,y,probabilities),...] where x,y are the top left corner of the 64*64 image.
    #probabilities is a tensor of 5 probabilities. lol
    #Gamma is the influence of the neighbours. Somewhere between 0 and 1.
    # FIlling the empty_img with the probabilities.
    empty_img= [[[]for i in range(1024)] for j in range(1024)]
    for output in outputs:
        x,y,probabilities = output
        for i in range(64):
            for j in range(64):
                empty_img[x+i][y+j].append(probabilities)
    # finding what value to assign to each pixel.
    with_neighbours_img = [[empty_img[j][i] for i in range(1024)] for j in range(1024)]
    processed_img = [[0 for i in range(1024)] for j in range(1024)]
    for i in range(1024):
        for j in range(1024):
            neihbours = [(-1,0),(1,0),(0,-1),(0,1)]
            for dx,dy in neihbours:
                if i+dx>=0 and i+dx<1024 and j+dy>=0 and j+dy<1024:
                    for k in empty_img[i+dx][j+dy]:
                        with_neighbours_img[i][j].append([l*gamma for l in k])
                    
            #find the most probable value for the pixel. and assign the index to provessed_img[i][j]
            #add all the probabilities and find the max.
            probs= [0,0,0,0,0]
            for k in with_neighbours_img[i][j]:
                for l in range(5):
                    probs[l]+=k[l]
            processed_img[i][j] = probs.index(max(probs))
    print(with_neighbours_img[64][64])
    print(processed_img[64][64])
    #return processed_img
    return processed_img
            #finding the most probable value for the pixel.
#test function with outputs of len 300 and gamma = 0.5
def test_post_process():
    import random
    def generate_probability_distribution():
    # Ensure the first value is at least 0.3
        first_prob = np.random.uniform(0.3, 1.0)  
    
    # Generate remaining 4 values randomly
        remaining_probs = np.random.rand(4)  
        remaining_probs /= remaining_probs.sum()  # Normalize to sum to 1
        remaining_probs *= (1 - first_prob)  # Scale to sum to (1 - first_prob)

    # Combine first value with the rest
        distribution = np.insert(remaining_probs, 0, first_prob)  
        return distribution.tolist()
    outputs = []
    for i in range(300):
        x = random.randint(0,1024-64)
        y = random.randint(0,1024-64)
        #probabilities need to sum to 1.
        probabilities = generate_probability_distribution()
        outputs.append((x,y,probabilities))
    processed_img = post_process(outputs)
    print(processed_img)
    return processed_img
#new function to convert the processesed to polygons
test_post_process()


    
    

    
